In [114]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
import html5lib
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

In [115]:
html = requests.get('https://jail.desotosheriff.org/DCN/inmates-released').text
soup = BeautifulSoup(html, 'html5lib')

In [116]:
def get_table(tbrow): 
    
    name = tbrow.find("td", {'id': re.compile('gvInmates_tccell')}).text
    name = re.sub('[\n\t]', "", name)
    
    age = tbrow.findAll('td', {'class': re.compile('dxgv*')})[1].text
    race = tbrow.findAll('td', {'class': re.compile('dxgv*')})[2].text
    sex = tbrow.findAll('td', {'class': re.compile('dxgv*')})[3].text
    admit = tbrow.findAll('td', {'class': re.compile('dxgv*')})[4].text
    release = tbrow.findAll('td', {'class': re.compile('dxgv*')})[5].text
    tij =  tbrow.findAll('td', {'class': re.compile('dxgv*')})[6].text
    
    inmate_row = {'name': name,
                    'age': age,
                    'race': race,
                    'sex': sex,
                    'admit': admit,
                    'release': release,
                    'tij': tij}
    return inmate_row

In [117]:
url = "https://jail.desotosheriff.org/DCN/inmates-released"

# create a new Firefox session
driver = webdriver.Firefox()
driver.get(url)

In [118]:
soups = []

In [119]:
for i in range(178):
    try:
        html = driver.page_source
        bsObj = BeautifulSoup(html, "html5lib")
        soups.append(bsObj)
        wait = WebDriverWait(driver, 45)
        el = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="gvInmates_DXPagerBottom"]/a[@class="dxp-button dxp-bi"]/img[@alt="Next"]')))    
        driver.execute_script("arguments[0].click();", el)  
        time.sleep(1)
    except: 
        print("done")



In [121]:
soups[176]

<html class="dxFirefox dxMacOSPlatform dxBrowserVersion-63"><head><link href="/DCN/DXR.axd?r=0_1993,0_2084,1_50,1_53,1_51,0_1997,0_2089,1_17,0_1986,1_16,0_1991,0_2214,0_2219-_iS0g" rel="stylesheet" type="text/css"/><title>
	DeSoto County Jail - Released Inmates
</title><meta charset="utf-8"/><meta content="noindex" name="robots"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link href="favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="https://cdnjs.cloudflare.com/ajax/libs/bootswatch/3.3.6/cosmo/bootstrap.min.css" rel="stylesheet"/><link href="styles/main.css" rel="stylesheet"/><link href="styles/grid.css" rel="stylesheet"/><link href="styles/banner-alert.css" rel="stylesheet"/><style id="ServerStyle" type="text/css">body {
background:White;
}

a:link {
color:#275DFF;
}

a:visited {
color:#1C42B5;
}

#BannerAlert, #BannerAlert:hover, .banner-alert {
color:#FFFFFF;
}

.navbar-custom {
color:#FFFFFF;
border-color:#196CC8 !important;
background:#278

In [122]:
table_rows = []
for pg in soups:
    for tabrow in pg.findAll('tr', {'id': re.compile('gvInmates_DXDataRow*')}):
        table_rows.append(get_table(tabrow))

In [123]:
table = pd.DataFrame(table_rows)

In [124]:
table.to_csv('test.csv')